In [12]:
!pip install nvcc4jupyter

In [13]:
%load_ext nvcc4jupyter

The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


In [16]:
%%cuda
#include <cuda.h>
#include <cuda_runtime.h>
#include <iostream>

#define N 100 // Dimensión de las matrices (100x100)
using namespace std;

// DEVICE: Kernel para multiplicar dos matrices
__global__ void matrixMultiply(float* A, float* B, float* C, int n) {
    // Índices de fila y columna
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    // Solo procesar elementos dentro de la matriz
    if (row < n && col < n) {
        float value = 0;
        // Realizar la suma de los productos de la fila y la columna
        for (int k = 0; k < n; k++) {
            value += A[row * n + k] * B[k * n + col];
        }
        // Guardar el resultado en la matriz de salida
        C[row * n + col] = value;
    }
}

// HOST: Función principal
int main() {
    // Dimensiones de las matrices
    int size = N * N;

    // Reservar memoria en el host
    float* A = new float[size];
    float* B = new float[size];
    float* C = new float[size]; // Matriz resultado

    // Inicializar matrices A y B
    for (int i = 0; i < size; i++) {
        A[i] = 1.0; // Llenar con 1.0 por simplicidad
        B[i] = 1.0; // Llenar con 1.0 por simplicidad
    }

    // Reservar memoria en el dispositivo
    float *d_A, *d_B, *d_C;
    cudaMalloc((void**)&d_A, size * sizeof(float));
    cudaMalloc((void**)&d_B, size * sizeof(float));
    cudaMalloc((void**)&d_C, size * sizeof(float));

    // Copiar datos del host al dispositivo
    cudaMemcpy(d_A, A, size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, size * sizeof(float), cudaMemcpyHostToDevice);

    // Configurar la rejilla y los bloques
    dim3 threadsPerBlock(16, 16); // 16x16 hilos por bloque
    dim3 blocksPerGrid((N + threadsPerBlock.x - 1) / threadsPerBlock.x,
                       (N + threadsPerBlock.y - 1) / threadsPerBlock.y);

    // Ejecutar el kernel
    matrixMultiply<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C, N);

    // Sincronizar
    cudaDeviceSynchronize();

    // Copiar resultado de vuelta al host
    cudaMemcpy(C, d_C, size * sizeof(float), cudaMemcpyDeviceToHost);

    // Imprimir la matriz resultado (solo los primeros 10x10 elementos)
    cout << "Matriz Resultado:" << endl;
    for (int i = 0; i < 10; i++) {
        for (int j = 0; j < 10; j++) {
            cout << C[i * N + j] << " ";
        }
        cout << endl;
    }

    // Liberar memoria en el dispositivo
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    // Liberar memoria en el host
    delete[] A;
    delete[] B;
    delete[] C;

    return 0;
}


Matriz Resultado:
100 100 100 100 100 100 100 100 100 100 
100 100 100 100 100 100 100 100 100 100 
100 100 100 100 100 100 100 100 100 100 
100 100 100 100 100 100 100 100 100 100 
100 100 100 100 100 100 100 100 100 100 
100 100 100 100 100 100 100 100 100 100 
100 100 100 100 100 100 100 100 100 100 
100 100 100 100 100 100 100 100 100 100 
100 100 100 100 100 100 100 100 100 100 
100 100 100 100 100 100 100 100 100 100 

